# Bagging and Ensembling with Random Forest

<b> Goals </b>

- Random Forest model: the ins and outs and how it relates to Decision Trees. Our first **black box** model.
- The concept of bagging and ensembling in the context of machine learning and specifically the Random Forest model.
- Compare and contrast Decision Trees with Random Forest using both the regression and classification of each model.

## Ensembling


Let's pretend that instead of building a single model to solve a binary classification problem, you created **five independent models**, and each model was correct about 70% of the time. If you combined these models into an "ensemble" and used their majority vote as a prediction, how often would the ensemble be correct? Think wisdom of the crowds.

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
%matplotlib inline

In [ ]:


# set a seed for reproducibility
np.random.seed(1234)

# generate 1000 random numbers (between 0 and 1) for each model, representing 1000 observations
mod1 = np.random.rand(1000)
mod2 = np.random.rand(1000)
mod3 = np.random.rand(1000)
mod4 = np.random.rand(1000)
mod5 = np.random.rand(1000)

# each model independently predicts 1 (the "correct response") if random number was at least 0.3
preds1 = np.where(mod1 > 0.3, 1, 0)
preds2 = np.where(mod2 > 0.3, 1, 0)
preds3 = np.where(mod3 > 0.3, 1, 0)
preds4 = np.where(mod4 > 0.3, 1, 0)
preds5 = np.where(mod5 > 0.3, 1, 0)

# print the first 20 predictions from each model
print preds1[:20]
print preds2[:20]
print preds3[:20]
print preds4[:20]
print preds5[:20]

In [ ]:
# average the predictions and then round to 0 or 1
ensemble_preds = np.round((preds1 + preds2 + preds3 + preds4 + preds5)/5.0).astype(int)

# print the ensemble's first 20 predictions
print ensemble_preds[:20]

In [ ]:
# how accurate was each individual model?
print preds1.mean()
print preds2.mean()
print preds3.mean()
print preds4.mean()
print preds5.mean()

In [ ]:
# how accurate was the ensemble?
print ensemble_preds.mean()

**Note:** As you add more models to the voting process, the probability of error decreases, which is known as [Condorcet's Jury Theorem](http://en.wikipedia.org/wiki/Condorcet%27s_jury_theorem).

### What is ensembling?

<br>

**Ensemble learning (or "ensembling")** is the process of combining several predictive models in order to produce a combined model that is more accurate than any individual model. Ensemble models are basically models made up of other models. 

- **Regression:** take the average of the predictions
- **Classification:** take a vote and use the most common prediction, or take the average of the predicted probabilities

For ensembling to work well, the models must have the following characteristics:

- **Accurate:** they outperform the null model
- **Independent:** their predictions are generated using different processes

**The big idea:** If you have a collection of individually imperfect (and independent) models, the "one-off" mistakes made by each model are probably not going to be made by the rest of the models, and thus the mistakes will be discarded when averaging the models.

There are two basic **methods for ensembling:**

- Manually ensemble your individual models
- Use a model that ensembles for you

### Manual Ensembling
![Machine learning flowchart](images/crowdflower_ensembling.jpg)

*Machine learning flowchart created by the [winner](https://github.com/ChenglongChen/Kaggle_CrowdFlower) of Kaggle's [CrowdFlower competition](https://www.kaggle.com/c/crowdflower-search-relevance)*

What makes a good manual ensemble?

- Different types of **models**
- Different combinations of **features**
- Different **tuning parameters**

### Comparing manual ensembling with a single model approach

**Advantages of manual ensembling:**

- Increases predictive accuracy
- Easy to get started

**Disadvantages of manual ensembling:**

- Decreases interpretability
- Takes longer to train
- Takes longer to predict
- More complex to automate and maintain
- Small gains in accuracy may not be worth the added complexity

## Random Forest

As you may have already guessed, Random Forest is related to Decision Trees. Knowing what we know about Decision Trees, what do you think Random Forest does?


- Decision Tree is singular while Random Forest is plural. Instead of one decision, we have multiple, a whole forest of decisions.  
- Each tree in the Random Forest gets a vote on deciding the outcome.
- Random Forest is a type of Ensembling because these sub models are Decision Trees. Random Forest generates many Decision Trees and combines them to generate a single prediction through a voting process.
- Random Forest usually better than Decision Trees because it's not as vulnerable to overfitting. This is why Random Forest has become such a popular algorithm for the data scientists.
- Like Decision Trees, you can set the parameters such as maximum number of features. Random Forest also uses the same attributes to determine the split such as gini and [entropy](http://www.saedsayad.com/decision_tree.htm).
- But perhaps the most significant parameter we have to set is the number of trees or estimators in our model, this means we have to "tune" this parameter in order to optimize our model.
- Regression application.


## Bagging

Random forest models do not just create a whole bunch of trees from the same data. They use <b>bagging</b> and a random selection of features to generate different trees. If they didn't use this, then the trees would all be very similar if not the same, which would almost eliminate the point of Random Forest.
<br><br>
<b>Bagging:</b>  General purpose procedure for reducing the variance of a machine learning method. Bagging is short for bootstrap aggregation, meaning the aggregation of bootstrap samples. What is a **bootstrap sample**? A random sample with replacement. 

<br>
Each tree selects a sample of observations/events with replacement to build the training set. Replacement means it chooses the same observation multiple times — only an issue with very small sample datasets. Observation is put "back in the bag" for future use. 

**How does bagging work for Random Forest?**

1. Grow B trees using B bootstrap samples from the training data.
2. Train each tree on its bootstrap sample and make predictions.
3. Combine the predictions:
    - Average the predictions for **regression trees**
    - Take a vote for **classification trees**

Notes:

- **Each bootstrap sample** should be the same size as the original training set.
- **B** should be a large enough value that the error seems to have "stabilized".
- The trees are **grown deep** so that they have low bias/high variance. Means we do not set max_depth.

Bagging increases predictive accuracy by **reducing the variance**, similar to how cross-validation reduces the variance associated with train/test split (for estimating out-of-sample error) by splitting many times an averaging the results.

Random Forest uses a random selection of features for each split. This means for each time it has to perform a split or generate a rule, it is only looking at this random sample of the features as possibilities to generate that rule. This will help avoid the similarity problem because the trees will not be built with the same set features at every point. The number of features is usually determined by $\sqrt{x}$ features are used for classifiers and $x/3$ for regression.
<br><br>
**Whats the point?**

- Suppose there is **one very strong feature** in the data set. When using bagged trees, most of the trees will use that feature as the top split, resulting in an ensemble of similar trees that are **highly correlated**.
- Averaging highly correlated quantities does not significantly reduce variance (which is the entire goal of bagging).
- By randomly leaving out candidate features from each split, **Random Forests "decorrelates" the trees**, such that the averaging process can reduce the variance of the resulting model.

### Pros and Cons of Random Forest

**Pros:**

- No feature preparation such as scaling.
- Effective, beats most ML algorithms.
- Fast, can train using data very large in a reasonable amount of time.
- It's hard to build a bad RF model.

**Cons:**

- Can grow to be way too large and complex. Models can literally take dozens of MBs in space.
- Not interpretable like a Decision Tree.
- Slower at training and predicting than Decision Trees.
- Not great for when the focus is on probabilities.
- Black box model qualities because there lacks some transparency in the modeling process. It's hard to gain insight into how it comes to conclusions.Requires a bit of faith from the data scientist. 


### Manually implementing bagged decision trees (with B=10)

We're going to use the vehicles dataset to demonstrate bagging for a regresion project.

In [ ]:
# set a seed for reproducibility
np.random.seed(1)

# create an array of 1 through 20
nums = np.arange(1, 21)
print nums

# sample that array 20 times with replacement
print np.random.choice(a=nums, size=20, replace=True)

In [ ]:
# read in and prepare the vehicle training data

url = '../../data/vehicles_train.csv'
train = 
train['vtype'] = 
train

Create random sets of index values for random sampling

In [ ]:
# set a seed for reproducibility
np.random.seed(123)

# create ten bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=14, size=14, replace=True) for _ in range(10)]
samples

In [ ]:
# show the rows for the first decision tree


In [ ]:
# read in and prepare the vehicle testing data
url = '../../data/vehicles_test.csv'
test = 
test['vtype'] = 
test

In [ ]:


# grow each tree deep
treereg = 

# list for storing predicted price from each tree
predictions = []

# define testing data
X_test = 
y_test = 

# grow one tree for each bootstrap sample and make predictions on testing data
for sample in samples:
    X_train = 
    y_train = 
    
    y_pred = 
    predictions

# convert predictions from list to NumPy array
predictions = 
predictions

10 rows of prediction for each of the ten samples. 3 columns for each car in the in test dataset.

In [ ]:
# average predictions


In [ ]:
# calculate RMSE

y_pred = 
np.sqrt()

Let's calculate RMSE by training the whole dataset on a Decision Tree Regressor


In [ ]:
treereg = 
treereg
preds = 
#RMSE
np.sqrt()

### Bagged decision trees in scikit-learn (with estimators=500)

In [ ]:
# define the training and testing sets
X_train = 
y_train = 
X_test = 
y_test = 

In [ ]:
# instruct BaggingRegressor to use DecisionTreeRegressor as the "base estimator"

bagreg = 


In [ ]:
# fit and predict
bagreg
y_pred = 
y_pred

In [ ]:
# calculate RMSE
np.sqrt()

In [ ]:
#Use RandomForest regression to caculate RMSE



#Intialize
rfe = 

#Fit


#Predictions
y_pred = 

# calculate RMSE
np.sqrt()

### Estimating out-of-sample error

For bagged models, out-of-sample error can be estimated without using train/test split or cross-validation

On average, each bagged tree uses about **two-thirds** of the observations. For each tree, the **remaining observations** are called "out-of-bag" observations.

In [ ]:
# show the first bootstrap sample


In [ ]:
# show the "in-bag" observations for each sample
for sample in samples:
    print set(sample)

In [ ]:
# show the "out-of-bag" observations for each sample
for sample in samples:
    print sorted(set(range(14)) - set(sample))

How to calculate **"out-of-bag error":**

1. For every observation in the training data, predict its response value using **only** the trees in which that observation was out-of-bag. Average those predictions (for regression) or take a vote (for classification).
2. Compare all predictions to the actual response values in order to compute the out-of-bag error.

When n_estimators is sufficiently large, the **out-of-bag error** is an accurate estimate of **out-of-sample error**.

In [ ]:
# compute the out-of-bag R-squared score (not MSE, unfortunately!) for n_estimators=500


### Estimating feature importance

Bagging increases **predictive accuracy**, but decreases **model interpretability** because it's no longer possible to visualize the tree to understand the importance of each feature.

However, we can still obtain an overall summary of **feature importance** from bagged models:

- **Bagged regression trees:** calculate the total amount that **MSE** is decreased due to splits over a given feature, averaged over all trees
- **Bagged classification trees:** calculate the total amount that **Gini index** is decreased due to splits over a given feature, averaged over all trees

## Building and Tuning Decision Trees and Random Forest Models

- Major League Baseball player data from 1986-87: [data](https://github.com/justmarkham/DAT8/blob/master/data/hitters.csv), [data dictionary](https://cran.r-project.org/web/packages/ISLR/ISLR.pdf) (page 7)
- Each observation represents a player
- **Goal:** Predict player salary

In [ ]:
# read in the data
path = '../../data/hitters.csv'
hitters = 

# remove rows with missing values


#View data
hitters.head()

In [ ]:
#lower case column names

hitters.columns = hitters.columns.str.lower()

In [ ]:
hitters.head(1)

In [ ]:
# encode categorical variables as integers
hitters['league'] = pd.factorize(hitters.league)[0]
hitters['division'] = pd.factorize(hitters.division)[0]
hitters['newleague'] = pd.factorize(hitters.newleague)[0]
hitters.head()

In [ ]:
#Scatter plot of years versus hits colored by salary

plt.style.use("fivethirtyeight")
hitters.plot(kind='scatter', x='years', y='hits',
             c='salary', cmap=plt.cm.get_cmap('RdBu'), s = 100, alpha = .7,
             xlim=(0, 25), ylim=(0, 250), figsize=(9, 7));

In [ ]:
# define features: exclude career statistics (which start with "C") and the response (Salary)
feature_cols = [h for h in hitters.columns if h[0] != 'c' and h != 'salary']
feature_cols

In [ ]:
# define X and y
X = hitters[feature_cols]
y = hitters.salary

### Predicting salary with a decision tree

Find the best max_depth for a decision tree using cross-validation:

In [ ]:
# list of values from 1 - 21 to try for max_depth
max_depth_range = 

# list to store the average RMSE for each value of max_depth
RMSE_scores = []

# use 5-fold cross-validation with each value of max_depth

for depth in max_depth_range:
    
    

In [ ]:
# plot max_depth (x-axis) versus RMSE (y-axis)
plt.figure(figsize=(10, 7))
plt.plot(max_depth_range, RMSE_scores, linewidth = 6)
plt.xlabel('max_depth')
plt.ylabel('RMSE')
plt.show()

Whats the best max_depth?

In [ ]:
# show the best RMSE and the corresponding max_depth


In [ ]:
# max_depth=2 was best, so fit a tree using that parameter
treereg = 


In [ ]:
# compute feature importances
pd.DataFrame({'feature':feature_cols, 
              'importance':treereg.feature_importances_}).sort_values('importance', ascending = False)

### Predicting salary with a Random Forest

In [ ]:

#Intialize
rfreg = RandomForestRegressor()
#Show model details
rfreg

#### Tuning n_estimators

One important tuning parameter is **n_estimators**, which is the number of trees that should be grown. It should be a large enough value that the error seems to have "stabilized". We want to reach a point of diminishing marginal returns.

In [ ]:
# list of values from 10 to 210 in intervals of 10 to try for n_estimators
estimator_range = 

# list to store the average RMSE for each value of n_estimators
RMSE_scores = 

# use 5-fold cross-validation with each value of n_estimators. Will take a while.

for estimator in estimator_range:
    
    

In [ ]:
# plot n_estimators (x-axis) versus RMSE (y-axis)
plt.figure(figsize=(10, 7))
plt.plot(estimator_range, RMSE_scores)
plt.xlabel('n_estimators')
plt.ylabel('RMSE');

In [ ]:
#Best estimator


#### Tuning max_features

The other important tuning parameter is **max_features**, which is the number of features that should be considered at each split.

In [ ]:
# list of values to try for max_features
feature_range = 

# list to store the average RMSE for each value of max_features
RMSE_scores = []

# use 5-fold cross-validation with each value of max_features (WARNING: SLOW!)
for feature in feature_range:
    
    

In [ ]:
# plot max_features (x-axis) versus RMSE (y-axis)
plt.figure(figsize=(10, 7))
plt.plot(feature_range, RMSE_scores)
plt.xlabel('max_features')
plt.ylabel('RMSE');

In [ ]:
# show the best RMSE and the corresponding max_features


#### Fitting a Random Forest with the best parameters

In [ ]:
# max_features=6 is best and n_estimators=150 is sufficiently large
rfreg = RandomForestRegressor()


In [ ]:
# compute feature importances
pd.DataFrame({'feature':feature_cols,
              'importance':rfreg.feature_importances_}).sort_values('importance', ascending = False)

In [ ]:
# compute the out-of-bag R-squared score


### Reducing X to its most important features

In [ ]:
# check the shape of X
X.shape

In [ ]:
# set a threshold for which features to include
print 




In [ ]:
#Use the mean threshold


In [ ]:
# create a new feature matrix that only includes important features
X_important = 

In [ ]:
# check the RMSE for a Random Forest that only includes important features
rfreg = RandomForestRegressor(n_estimators=150, max_features=3, random_state=1)
scores = cross_val_score(rfreg, X_important, y, cv=10, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

In [ ]:
# check the RMSE for a Random Forest that includes ALL features ( for comparison )
rfreg = RandomForestRegressor(n_estimators=150, max_features=3, random_state=1)
scores = cross_val_score(rfreg, X, y, cv=10, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

## Visualizing Decision Trees versus Random Forest

In [ ]:


#Generate fake data that is 400 x 2.
data = make_classification(n_samples=400, n_features=2, n_informative=2, n_redundant=0, 
                    class_sep=.74, random_state = 28)

df = pd.DataFrame(data[0], columns=["feature1", "feature2"])
#Add target variable to df 
df["target"] = data[1]

#Call scatter plot of feature1 vs feature2 with color-encoded target variable

plt.figure(figsize=(11, 8))
#Color encode target variable
colors = df.target.map({0:"b", 1:"r"})
plt.scatter(df.feature1, df.feature2, c = colors, s = 100, alpha=.5);

In [ ]:
#Assign X and y
X = df.drop("target", axis = 1)
y = df.target

#Fit a Decision Tree model with 5 max_depth on the data.

dt = DecisionTreeClassifier(max_depth=5)

dt.fit(X, y)

In [ ]:
#Decision boundary function
def plot_decision_boundary(model, X, y):
    X_max = X.max(axis=0)
    X_min = X.min(axis=0)
    xticks = np.linspace(X_min[0], X_max[0], 100)
    yticks = np.linspace(X_min[1], X_max[1], 100)
    xx, yy = np.meshgrid(xticks, yticks)
    ZZ = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = ZZ >= 0.5
    Z = Z.reshape(xx.shape)
    plt.rcParams["figure.figsize"] = (10,7)
    fig, ax = plt.subplots()
    ax = plt.gca()
    ax.contourf(xx, yy, Z, cmap=plt.cm.bwr, alpha=0.2)
    ax.scatter(X[:,0], X[:,1], c=y, alpha=0.4, s = 50)

In [ ]:
#Feed dt model, features and colors
plot_decision_boundary(dt, X.values, colors);

Fit RF model on data and visualize it

In [ ]:


#Use estimators = 30
rf = RandomForestClassifier(n_estimators = 50)

#Fit model
rf.fit(X, y)

In [ ]:
#Feed rf model, features and colors
plot_decision_boundary(rf, X.values, colors);

## Resources

- http://blog.datadive.net/interpreting-random-forests/
- http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/
- https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb
- https://aysent.github.io/2015/11/08/random-forest-leaf-visualization.html
- https://medium.com/rants-on-machine-learning/the-unreasonable-effectiveness-of-random-forests-f33c3ce28883
- https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/
- http://paolaelefante.com/2016/03/a-small-guide-to-random-forest-part-2/
- https://www.youtube.com/watch?v=QHOazyP-YlM
- https://www.youtube.com/watch?v=loNcrMjYh64
- https://machinelearningmastery.com/bagging-and-random-forest-ensemble-algorithms-for-machine-learning/
- https://machinelearningmastery.com/implement-bagging-scratch-python/
- https://towardsdatascience.com/enchanted-random-forest-b08d418cb411
- https://github.com/Miguel75An/Random-Forests-with-Iris-Data

## Classifying Employee Churn with Decision Trees and Random Forest

In class exercise in which we will apply both the Decision Trees and Random Forest models to the Employee Churn data. We'll compare and contrast the performances of both models using this dataset.

In [ ]:
df = pd.read_csv('../../data/HR_comma_sep.csv')

df.head()